# Evolver Loop 1 Analysis

Analyze data patterns and identify feature engineering opportunities based on evaluator feedback.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\nTarget distribution:")
print(train['NObeyesdad'].value_counts(normalize=True).round(4))

Train shape: (20758, 18)
Test shape: (13840, 17)

Target distribution:
NObeyesdad
Obesity_Type_III       0.1949
Obesity_Type_II        0.1565
Normal_Weight          0.1485
Obesity_Type_I         0.1402
Insufficient_Weight    0.1215
Overweight_Level_II    0.1215
Overweight_Level_I     0.1169
Name: proportion, dtype: float64


In [2]:
# Basic feature engineering to explore patterns
def engineer_features(df):
    df = df.copy()
    
    # BMI - critical for obesity prediction
    df['BMI'] = df['Weight'] / (df['Height'] ** 2)
    
    # Weight/Height ratio
    df['Weight_Height_Ratio'] = df['Weight'] / df['Height']
    
    # Age groups
    df['Age_Group'] = pd.cut(df['Age'], 
                            bins=[0, 18, 30, 45, 60, 100], 
                            labels=['0-18', '19-30', '31-45', '46-60', '60+'])
    
    # Simple interactions
    df['Age_Height'] = df['Age'] * df['Height']
    df['Age_Weight'] = df['Age'] * df['Weight']
    df['Height_Weight'] = df['Height'] * df['Weight']
    
    # Lifestyle combinations
    df['FCVC_NCP'] = df['FCVC'] * df['NCP']  # Frequency of consumption * Number of main meals
    df['CH2O_FAF'] = df['CH2O'] * df['FAF']  # Water consumption * Physical activity
    df['FAF_TUE'] = df['FAF'] * df['TUE']    # Physical activity * Time using tech
    
    return df

train_fe = engineer_features(train)

# Analyze BMI distribution by target
print("BMI statistics by target class:")
bmi_stats = train_fe.groupby('NObeyesdad')['BMI'].agg(['mean', 'std', 'min', 'max', 'count']).round(2)
print(bmi_stats)

# BMI categories (WHO standards)
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    elif bmi < 35:
        return 'Obese_I'
    elif bmi < 40:
        return 'Obese_II'
    else:
        return 'Obese_III'

train_fe['BMI_Category'] = train_fe['BMI'].apply(categorize_bmi)

print("\nBMI Category distribution:")
print(train_fe['BMI_Category'].value_counts())

print("\nTarget vs BMI Category cross-tab:")
bmi_target = pd.crosstab(train_fe['BMI_Category'], train_fe['NObeyesdad'], normalize='index').round(3)
print(bmi_target)

BMI statistics by target class:
                      mean   std    min    max  count
NObeyesdad                                           
Insufficient_Weight  17.58  1.39  12.87  36.78   2523
Normal_Weight        22.00  2.19  14.63  32.88   3082
Obesity_Type_I       32.15  2.51  16.60  46.81   2910
Obesity_Type_II      36.52  2.11  24.05  46.22   3248
Obesity_Type_III     41.78  2.67  18.18  55.00   4046
Overweight_Level_I   26.06  1.77  15.76  35.56   2427
Overweight_Level_II  28.19  1.84  21.51  36.74   2522

BMI Category distribution:
BMI_Category
Overweight     4740
Obese_II       3713
Normal         3526
Obese_III      3253
Obese_I        3118
Underweight    2408
Name: count, dtype: int64

Target vs BMI Category cross-tab:
NObeyesdad    Insufficient_Weight  Normal_Weight  Obesity_Type_I  \
BMI_Category                                                       
Normal                      0.086          0.776           0.002   
Obese_I                     0.000          0.002        

In [3]:
# Analyze feature correlations with target
from sklearn.preprocessing import LabelEncoder

# Encode target for correlation analysis
le_target = LabelEncoder()
target_encoded = le_target.fit_transform(train_fe['NObeyesdad'])

# Select numerical features for correlation
numerical_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 
                     'BMI', 'Weight_Height_Ratio', 'Age_Height', 'Age_Weight', 'Height_Weight',
                     'FCVC_NCP', 'CH2O_FAF', 'FAF_TUE']

correlations = {}
for feature in numerical_features:
    corr = np.corrcoef(train_fe[feature], target_encoded)[0, 1]
    correlations[feature] = abs(corr)

# Sort by absolute correlation
corr_series = pd.Series(correlations).sort_values(ascending=False)
print("Feature correlations with target (absolute):")
print(corr_series.round(4))

Feature correlations with target (absolute):
BMI                    0.4639
Weight_Height_Ratio    0.4543
Weight                 0.4318
Age_Weight             0.4193
Height_Weight          0.4016
Age_Height             0.2902
Age                    0.2830
CH2O                   0.1871
FAF_TUE                0.0977
FAF                    0.0966
NCP                    0.0912
TUE                    0.0760
Height                 0.0608
FCVC                   0.0411
CH2O_FAF               0.0300
FCVC_NCP               0.0205
dtype: float64


In [4]:
# Analyze categorical features
from scipy.stats import chi2_contingency

categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 
                       'SMOKE', 'SCC', 'CALC', 'MTRANS', 'Age_Group']

chi2_results = {}
for feature in categorical_features:
    contingency = pd.crosstab(train_fe[feature], train_fe['NObeyesdad'])
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    chi2_results[feature] = {'chi2': chi2, 'p_value': p_value}
    print(f"{feature}: chi2={chi2:.2f}, p_value={p_value:.4f}")

# Show most predictive categorical patterns
print("\n=== family_history_with_overweight vs Target ===")
print(pd.crosstab(train_fe['family_history_with_overweight'], train_fe['NObeyesdad'], normalize='index').round(3))

print("\n=== CAEC (Consumption of high caloric food) vs Target ===")
print(pd.crosstab(train_fe['CAEC'], train_fe['NObeyesdad'], normalize='index').round(3))

print("\n=== MTRANS (Transportation) vs Target ===")
print(pd.crosstab(train_fe['MTRANS'], train_fe['NObeyesdad'], normalize='index').round(3))

Gender: chi2=7953.77, p_value=0.0000
family_history_with_overweight: chi2=6423.32, p_value=0.0000
FAVC: chi2=1553.63, p_value=0.0000
CAEC: chi2=6897.33, p_value=0.0000
SMOKE: chi2=216.30, p_value=0.0000
SCC: chi2=1024.80, p_value=0.0000
CALC: chi2=4013.08, p_value=0.0000
MTRANS: chi2=2349.08, p_value=0.0000
Age_Group: chi2=6317.37, p_value=0.0000

=== family_history_with_overweight vs Target ===
NObeyesdad                      Insufficient_Weight  Normal_Weight  \
family_history_with_overweight                                       
no                                            0.387          0.345   
yes                                           0.063          0.105   

NObeyesdad                      Obesity_Type_I  Obesity_Type_II  \
family_history_with_overweight                                    
no                                       0.013            0.001   
yes                                      0.168            0.191   

NObeyesdad                      Obesity_Type_III  O

In [5]:
# Analyze interactions between key features
# BMI is most important - let's see interactions with lifestyle factors

print("=== BMI Category vs Family History ===")
print(pd.crosstab([train_fe['BMI_Category'], train_fe['family_history_with_overweight']], 
                 train_fe['NObeyesdad'], normalize='index').round(3))

print("\n=== BMI Category vs CAEC ===")
print(pd.crosstab([train_fe['BMI_Category'], train_fe['CAEC']], 
                 train_fe['NObeyesdad'], normalize='index').round(3))

# Age group interactions
print("\n=== Age Group vs Family History ===")
print(pd.crosstab([train_fe['Age_Group'], train_fe['family_history_with_overweight']], 
                 train_fe['NObeyesdad'], normalize='index').round(3))

=== BMI Category vs Family History ===
NObeyesdad                                   Insufficient_Weight  \
BMI_Category family_history_with_overweight                        
Normal       no                                            0.102   
             yes                                           0.074   
Obese_I      no                                            0.000   
             yes                                           0.000   
Obese_II     no                                            0.000   
             yes                                           0.000   
Obese_III    no                                            0.000   
             yes                                           0.000   
Overweight   no                                            0.004   
             yes                                           0.002   
Underweight  no                                            0.925   
             yes                                           0.906   

NObeyesd

In [6]:
# Identify high-value feature engineering opportunities
print("=== KEY FINDINGS FOR FEATURE ENGINEERING ===\n")

print("1. BMI is the most important feature (correlation: {:.4f})".format(corr_series['BMI']))
print("   - WHO categories (Underweight, Normal, Overweight, Obese_I, Obese_II, Obese_III) are highly predictive")
print("   - BMI alone can distinguish many obesity classes\n")

print("2. Weight_Height_Ratio is second most important (correlation: {:.4f})".format(corr_series['Weight_Height_Ratio']))
print("   - Alternative to BMI, captures similar signal\n")

print("3. Family history is highly predictive (p_value: {:.4f})".format(chi2_results['family_history_with_overweight']['p_value']))
print("   - Strong interaction with BMI categories\n")

print("4. CAEC (high caloric food consumption) is significant (p_value: {:.4f})".format(chi2_results['CAEC']['p_value']))
print("   - Interacts with BMI and age\n")

print("5. Transportation mode (MTRANS) is significant (p_value: {:.4f})".format(chi2_results['MTRANS']['p_value']))
print("   - Public/Automobile vs Walking/Bike shows different patterns\n")

print("6. Lifestyle interactions show promise:")
print("   - FCVC_NCP (food consumption * meals) correlation: {:.4f}".format(corr_series['FCVC_NCP']))
print("   - CH2O_FAF (water * activity) correlation: {:.4f}".format(corr_series['CH2O_FAF']))
print("   - FAF_TUE (activity * tech use) correlation: {:.4f}\n".format(corr_series['FAF_TUE']))

print("7. Age groups show different obesity patterns, especially with family history")

# Recommend specific features to engineer
print("\n=== RECOMMENDED FEATURES FOR NEXT EXPERIMENT ===")
print("1. BMI_Category (WHO standards) - categorical")
print("2. Weight_Height_Ratio - numerical")
print("3. BMI_FamilyHistory interaction")
print("4. BMI_CAEC interaction")
print("5. Age_BMI interaction")
print("6. FCVC_NCP (food consumption frequency * meals)")
print("7. CH2O_FAF (water consumption * physical activity)")
print("8. FAF_TUE (physical activity * tech use)")
print("9. FamilyHistory_AgeGroup interaction")
print("10. More granular age bins (e.g., 10-year intervals)")

=== KEY FINDINGS FOR FEATURE ENGINEERING ===

1. BMI is the most important feature (correlation: 0.4639)
   - WHO categories (Underweight, Normal, Overweight, Obese_I, Obese_II, Obese_III) are highly predictive
   - BMI alone can distinguish many obesity classes

2. Weight_Height_Ratio is second most important (correlation: 0.4543)
   - Alternative to BMI, captures similar signal

3. Family history is highly predictive (p_value: 0.0000)
   - Strong interaction with BMI categories

4. CAEC (high caloric food consumption) is significant (p_value: 0.0000)
   - Interacts with BMI and age

5. Transportation mode (MTRANS) is significant (p_value: 0.0000)
   - Public/Automobile vs Walking/Bike shows different patterns

6. Lifestyle interactions show promise:
   - FCVC_NCP (food consumption * meals) correlation: 0.0205
   - CH2O_FAF (water * activity) correlation: 0.0300
   - FAF_TUE (activity * tech use) correlation: 0.0977

7. Age groups show different obesity patterns, especially with famil